In [1]:
from inference import SparseModel

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SparseModel()

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
QDRANT_URL='http://localhost:6333'
QDRANT_API_KEY=''
COLLECTION_NAME="fiqa"

In [12]:
import math
import numpy as np

In [79]:

# Split scores into 3 groups
# Group with highest score is assigned with value 3
# Group with second highest score is assigned with value 2
# Group with lowest score is assigned with value 1
def rescore_vector(vector):
    if len(vector) < 4:
        return np.ones_like(vector) * 2


    sorted_vector = np.array(np.sort(vector))
    diff = np.diff(sorted_vector)

    

    # Calculate biggest and second biggest difference
    gap_ids = np.argsort(diff)

    top_gap_id = gap_ids[-1]
    second_gap_id = gap_ids[-2]

    new_vector = np.ones_like(vector)

    new_vector[vector > sorted_vector[second_gap_id]] += 1
    new_vector[vector > sorted_vector[top_gap_id]] += 1

    return new_vector


In [5]:
def rescore_vector(vector: dict) -> dict:
    new_vector = {}

    sorted_vector = sorted(vector.items(), key=lambda x: x[1], reverse=True)

    for num, (idx, _value) in enumerate(sorted_vector):
        new_vector[idx] = 1. # math.log(1./(num + 1) + 1.) # * value
    return new_vector


In [7]:
def conver_sparse_vector(sparse_vector: dict) -> models.SparseVector:
    indices = []
    values = []

    for (idx, value) in sparse_vector.items():
        indices.append(int(idx))
        values.append(value)

    return models.SparseVector(
        indices=indices,
        values=values
    )

In [90]:
query = "What are the consequences of IRS “reclassification” on both employer and employee?"
query = "Is math an art or a science?"
query = "Should a retail trader bother about reading SEC filings?"

sparse_vector = next(model.encode([query]))
sparse_vector_qdrant = conver_sparse_vector(sparse_vector)


In [91]:
for (idx, value) in zip(sparse_vector_qdrant.indices, sparse_vector_qdrant.values):
    print(f"{model.invert_vocab[idx]}: {value}")

should: 0.024150477722287178
a: 0.029489323496818542
retail: 0.11483627557754517
trader: 0.11542164534330368
bother: 0.02308724820613861
about: 0.024228611961007118
reading: 0.05111318826675415
sec: 0.15624363720417023
filing: 0.0665680393576622
##s: 0.025189092382788658
?: 0.031020402908325195


In [92]:
for (idx, value) in zip(sparse_vector_qdrant.indices, rescore_vector(sparse_vector_qdrant.values)):
    print(f"{model.invert_vocab[idx]}: {value}")

should: 1.0
a: 1.0
retail: 2.0
trader: 2.0
bother: 1.0
about: 1.0
reading: 1.0
sec: 3.0
filing: 1.0
##s: 1.0
?: 1.0


In [23]:
std = np.std(sparse_vector_qdrant.values)
mean = np.mean(sparse_vector_qdrant.values)

new_values = np.ones_like(sparse_vector_qdrant.values) * 2

new_values[new_values < mean - std/2] = 3
new_values[new_values > mean + std/2] = 1


In [25]:
for (idx, value) in zip(sparse_vector_qdrant.indices, new_values):
    print(f"{model.invert_vocab[idx]}: {value}")

is: 1.0
math: 1.0
an: 1.0
art: 1.0
or: 1.0
a: 1.0
science: 1.0
?: 1.0


In [82]:
query = "Is math an art or a science?"
sv = next(model.encode([query]))
sv = conver_sparse_vector(sv)

for (idx, value) in zip(sv.indices, sv.values):
    print(f"{model.invert_vocab[idx]}: {value}")


is: 0.027513518929481506
math: 0.2162715643644333
an: 0.02588239312171936
art: 0.1304435133934021
or: 0.02149391360580921
a: 0.021214833483099937
science: 0.11642558127641678
?: 0.028545841574668884
